In [150]:
import pandas as pd
import numpy as np
import json
import math

In [151]:
class Node():
    def  __init__(self, 
                  s        = None, 
                  leftchild  = None,
                  rightchild = None,
                  f = None,
                  payoff = None):
        self.s        = s
        self.leftchild  = leftchild
        self.rightchild = rightchild
        self.f = f
        self.payoff = payoff

def print_tree(node, level=0, prefix="Root: "):
    if node is not None:
        print(" " * (level * 4) + prefix + str(node.s) + ", Option Price=" + str(node.f) + ", payoff=" + str(node.payoff))
        if node.leftchild is not None or node.rightchild is not None:
            print_tree(node.leftchild, level + 1, "Down-- ")
            print_tree(node.rightchild, level + 1, "Up-- ")

def dump(root: Node) -> str:
    def _to_dict(node) -> dict:    
        return {
            "stock price": node.s,
            "option price": node.f,
            "payoff": node.payoff,
            "down": (_to_dict(node.leftchild) if node.leftchild is not None else None),
            "up": (_to_dict(node.rightchild) if node.rightchild is not None else None)
        }
    if root == None:
        dict_repr = {}
    else:
        dict_repr = _to_dict(root)
    return json.dumps(dict_repr)

def generate_tree_ec(height, startPrice, u, d, strike, prob, rate):
    if height < 0:
        return None
    
    if height == 0:
        root = Node(s=startPrice, payoff=max(0, startPrice-strike))
        root.leftchild = generate_tree_ec(height - 1, startPrice*d, u, d, strike, prob, rate)
        root.rightchild = generate_tree_ec(height - 1, startPrice*u, u, d, strike, prob, rate)
        return root
    else:
        root = Node(s=startPrice, payoff=max(0, startPrice-strike))
        root.leftchild = generate_tree_ec(height - 1, startPrice*d, u, d, strike, prob, rate)
        root.rightchild = generate_tree_ec(height - 1, startPrice*u, u, d, strike, prob, rate)
        root.f = round(math.exp(-rate) * ((prob*root.rightchild.payoff) + ((1-prob)*root.leftchild.payoff)), 2)
        return root
def generate_tree_ep(height, startPrice, u, d, strike, prob, rate):
    if height < 0:
        return None
    
    if height == 0:
        root = Node(s=startPrice, payoff=max(0, strike-startPrice))
        root.leftchild = generate_tree_ep(height - 1, startPrice*d, u, d, strike, prob, rate)
        root.rightchild = generate_tree_ep(height - 1, startPrice*u, u, d, strike, prob, rate)
        return root
    else:
        root = Node(s=startPrice, payoff=max(0, strike-startPrice))
        root.leftchild = generate_tree_ep(height - 1, startPrice*d, u, d, strike, prob, rate)
        root.rightchild = generate_tree_ep(height - 1, startPrice*u, u, d, strike, prob, rate)
        root.f = round(math.exp(-rate) * ((prob*root.rightchild.payoff) + ((1-prob)*root.leftchild.payoff)), 2)
        return root

# Question 4

In [152]:
t = 1
sigma = 0.2
u = math.exp(sigma * math.sqrt(t))
d = 1/u
p = (math.exp(0.1) - d)/(u-d)
r = 0.1
print(sigma)
print(u)
print(d)
print(p)

0.2
1.2214027581601699
0.8187307530779818
0.7113485948425979


### European Call

In [156]:
print_tree(generate_tree_ec(2, 100, u, d, 100, p, r))

Root: 100, Option Price=14.25, payoff=0
    Down-- 81.87307530779819, Option Price=0.0, payoff=0
        Down-- 67.03200460356393, Option Price=None, payoff=0
        Up-- 100.0, Option Price=None, payoff=0
    Up-- 122.14027581601698, Option Price=31.66, payoff=22.14027581601698
        Down-- 100.0, Option Price=None, payoff=0
        Up-- 149.18246976412703, Option Price=None, payoff=49.182469764127035


### European Put

In [154]:
print_tree(generate_tree_ep(10, 100, u, d, 100, p, r))

Root: 100, Option Price=4.73, payoff=0
    Down-- 81.87307530779819, Option Price=8.61, payoff=18.12692469220181
        Down-- 67.03200460356393, Option Price=None, payoff=32.967995396436066
        Up-- 100.0, Option Price=None, payoff=0
    Up-- 122.14027581601698, Option Price=0.0, payoff=0
        Down-- 100.0, Option Price=None, payoff=0
        Up-- 149.18246976412703, Option Price=None, payoff=0
